The goal of this notebook is to extract the best individual from each replicate at the end of evolution in a list of EvoTSC runs, in order to be able to start a new set of simulations seeded with these individuals.

In [ ]:
import pathlib
import pickle
import re
import numpy as np

In [ ]:
import evotsc_lib
import evotsc_run

In [ ]:
exp_path = pathlib.Path('/Users/theotime/Desktop/evotsc/pci/main/')

In [ ]:
rep_dirs = sorted([d for d in exp_path.iterdir() if (d.is_dir() and d.name.startswith("rep"))])
params = evotsc_lib.read_params(rep_dirs[0])

In [ ]:
best_indivs_path = exp_path.joinpath(f'best_indivs_{exp_path.name}')
best_indivs_path.mkdir(exist_ok=True)

In [ ]:
for i_rep, rep_dir in enumerate(rep_dirs):
    save_files = [f for f in rep_dir.iterdir() if 'pop_gen' in f.name]

    if len(save_files) == 0:
        raise FileNotFoundError('No save files found: corrupted directory?.')
        
    last_save_path = sorted(save_files)[-1]
    
    start_gen = int(re.search(r'\d+', last_save_path.name).group(0))
    
    pop = evotsc_run.load_pop(last_save_path)
    
    pop.evaluate()
    fitnesses = [indiv.fitness for indiv in pop.individuals]
    best_indiv = pop.individuals[np.argmax(fitnesses)]
    
    save_path = best_indivs_path.joinpath(f'best_rep_{i_rep:02}_gen_{start_gen:07}.evotsc')
    with open(save_path, 'wb') as save_file:
        pickle.dump(best_indiv, save_file)